##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tipos de extensão

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/extension_type"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/extension_type.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/extension_type.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/extension_type.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Configuração

In [ ]:
!pip install -q tf_nightly
import tensorflow as tf
import numpy as np
from typing import Tuple, List, Mapping, Union, Optional
import tempfile

## Tipos de extensão

Os tipos definidos pelo usuário podem tornar os projetos mais legíveis, modulares e de fácil manutenção. No entanto, a maioria das APIs do TensorFlow tem suporte muito limitado aos tipos Python definidos pelo usuário. Isso inclui APIs de alto nível (como [Keras](https://www.tensorflow.org/guide/keras/overview), [tf.function](https://www.tensorflow.org/guide/function), [`tf.SavedModel`](https://www.tensorflow.org/guide/saved_model)) e APIs de baixo nível (como `tf.while_loop` e `tf.concat`). Os **tipos de extensão** do TensorFlow podem ser usados para criar tipos orientados a objetos definidos pelo usuário que funcionam perfeitamente com as APIs do TensorFlow. Para criar um tipo de extensão, basta definir uma classe Python com `tf.experimental.ExtensionType` como base e usar [anotações de tipo](https://www.python.org/dev/peps/pep-0484/) para especificar o tipo de cada campo.

In [ ]:
class TensorGraph(tf.experimental.ExtensionType):
  """A collection of labeled nodes connected by weighted edges."""
  edge_weights: tf.Tensor               # shape=[num_nodes, num_nodes]
  node_labels: Mapping[str, tf.Tensor]  # shape=[num_nodes]; dtype=any

class MaskedTensor(tf.experimental.ExtensionType):
  """A tensor paired with a boolean mask, indicating which values are valid."""
  values: tf.Tensor
  mask: tf.Tensor       # shape=values.shape; false for missing/invalid values.

class CSRSparseMatrix(tf.experimental.ExtensionType):
  """Compressed sparse row matrix (https://en.wikipedia.org/wiki/Sparse_matrix)."""
  values: tf.Tensor     # shape=[num_nonzero]; dtype=any
  col_index: tf.Tensor  # shape=[num_nonzero]; dtype=int64
  row_index: tf.Tensor  # shape=[num_rows+1]; dtype=int64

A classe base `tf.experimental.ExtensionType` funciona de maneira similar a [`typing.NamedTuple`](https://docs.python.org/3/library/typing.html#typing.NamedTuple) e [`@dataclasses.dataclass`](https://docs.python.org/3/library/dataclasses.html#dataclasses.dataclass) da biblioteca Python padrão. Além disso, ela adiciona automaticamente um construtor e métodos especiais (como `__repr__` e `__eq__`) com base nas anotações de tipo de campo.

Tipicamente, os tipos de extensão pertencem a uma de duas categorias:

- ***Estruturas de dados***, que agrupam uma coleção de valores relacionados e fornecem operações úteis com base nesses valores. As estruturas de dados podem ser bem gerais (como o exemplo `TensorGraph` acima) ou podem ser bastante personalizadas para um modelo específico.

- ***Tipos "tipo tensor'***, que especializam ou estendem o conceito do "Tensor". Os tipos desta categoria têm um `rank` (posto), um `shape` (format) e geralmente um `dtype` (tipo de dado), e faz sentido usá-los com as operações de Tensor (como `tf.stack`, `tf.add` ou `tf.matmul`). `MaskedTensor` e `CSRSparseMatrix` são exemplos de tipos "tipo tensor".

## APIs com suporte

As seguintes APIs do TensorFlow têm suporte aos tipos de extensão:

- **Keras**: os tipos de extensão podem ser usados como entradas e saídas para `Models` (modelos) e `Layers` (camadas) do Keras.
- **`tf.data.Dataset`**: os tipos de extensão podem ser incluídos em `Datasets` e retornados pelo dataset como `Iterators` (iteradores).
- **TensorFlow Hub**: os tipos de extensão podem ser usados como entradas e saídas de módulos do `tf.hub`.
- **SavedModel**: os tipos de extensão podem ser usados como entradas e saídas de funções `SavedModel`.
- **`tf.function`**: os tipos de extensão podem ser usados como argumentos e valores de retorno para funções encapsuladas com o decorador `@tf.function`.
- **Loops while**: os tipos de extensão podem ser usados como variáveis de loops em `tf.while_loop` e podem ser usados como argumentos e valores de retorno no corpo do loop while.
- **Condicionais**: os tipos de extensão podem ser selecionados condicionalmente ao usar `tf.cond` e `tf.case`.
- **`tf.py_function`**: os tipos de extensão podem ser usados como argumentos e valores de retorno para o argumento `func` de `tf.py_function`.
- **Operações de tensor**: os tipos de extensão podem ser usados para dar suporte à maioria das operações do TensorFlow que aceitam Tensores como entrada (como `tf.matmul`, `tf.gather` e `tf.reduce_sum`). Confira mais informações na seção "*Dispatch*" abaixo.
- **Estratégia de distribuição**: os tipos de extensão podem ser usados como valores por réplica.

Confira mais detalhes na seção "APIs do TensorFlow com suporte aos ExtensionTypes" abaixo.


## Requisitos


### Tipos de campo

Todos os campos — variáveis de instância — precisam ser declarados, e uma anotação de tipo precisa ser fornecida para cada campo. Há suporte às seguintes anotações de tipo:

Tipo | Exemplo
--- | ---
Inteiros Python | `i: int`
Floats Python | `f: float`
Strings Python | `s: str`
Booleanos Python | `b: bool`
`None` do Python | `n: None`
[Formatos de tensor](https://www.tensorflow.org/api_docs/python/tf/TensorShape) | `shape: tf.TensorShape`
[`dtype`s de tensor](https://www.tensorflow.org/api_docs/python/tf/dtypes/DType) | `dtype: tf.DType`
[Tensores](https://www.tensorflow.org/api_docs/python/tf/Tensor) | `t: tf.Tensor`
[Tipos de extensão](https://www.tensorflow.org/api_docs/python/tf/experimental/ExtensionType) | `mt: MyMaskedTensor`
[Tensores irregulares](https://www.tensorflow.org/api_docs/python/tf/RaggedTensor) | `rt: tf.RaggedTensor`
[Tensores esparsos](https://www.tensorflow.org/api_docs/python/tf/sparse/SparseTensor) | `st: tf.SparseTensor`
[Fatias indexadas](https://www.tensorflow.org/api_docs/python/tf/IndexedSlices) | `s: tf.IndexedSlices`
[Tensores opcionais](https://www.tensorflow.org/api_docs/python/tf/experimental/Optional) | `o: tf.experimental.Optional`
[Uniões de tipos](https://docs.python.org/3/library/typing.html#typing.Union) | `int_or_float: typing.Union[int, float]`
[Tuplas](https://docs.python.org/3/library/typing.html#typing.Tuple) | `params: typing.Tuple[int, float, tf.Tensor, int]`
[Tuplas de tamanho variável](https://docs.python.org/3/library/typing.html#typing.Tuple) | `lengths: typing.Tuple[int, ...]`
[Mapeamentos](https://docs.python.org/3/library/typing.html#typing.Mapping) | `tags: typing.Mapping[str, tf.Tensor]`
[Valores opcionais](https://docs.python.org/3/library/typing.html#typing.Optional) | `weight: typing.Optional[tf.Tensor]`

### Mutabilidade

Os tipos de extensão precisam ser imutáveis, o que garante que possam ser monitorados corretamente pelos mecanismos de criação de grafo do TensorFlow. Se você quiser mudar o valor do tipo de extensão, é melhor definir métodos que transformem valores. Por exemplo, em vez de definir um método `set_mask` para mudar um `MaskedTensor`, você pode definir um método `replace_mask` que retorne um novo `MaskedTensor`:

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  values: tf.Tensor
  mask: tf.Tensor

  def replace_mask(self, new_mask):
      self.values.shape.assert_is_compatible_with(new_mask.shape)
      return MaskedTensor(self.values, new_mask)

## Funcionalidades adicionadas por `ExtensionType`

A classe base `ExtensionType` tem as seguintes funcionalidades:

- Um construtor (`__init__`).
- Um método de representação que pode ser exibido via print (`__repr__`).
- Operadores de igualdade e desigualdade (`__eq__`).
- Um método de validação (`__validate__`).
- Imutabilidade imposta.
- Um `TypeSpec` aninhado.
- Suporte ao dispatch de API do tensor.

Confira mais informações sobre como personalizar essas funcionalidades na seção "Personalização de `ExtensionType`s" abaixo.

### Construtor

O construtor adicionado por `ExtensionType` recebe cada campo como um argumento com nome (na ordem indicada na definição da classe). Esse construtor verifica o tipo de todos os parâmetros e faz a conversão, quando necessário. Especificamente, os campos do `Tensor` são convertidos usando `tf.convert_to_tensor`; os campos de `Tuple` são convertidos em `tuple`s; e os campos de `Mapping` são convertidos em dicionários imutáveis.

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  values: tf.Tensor
  mask: tf.Tensor

# Constructor takes one parameter for each field.
mt = MaskedTensor(values=[[1, 2, 3], [4, 5, 6]],
                  mask=[[True, True, False], [True, False, True]])

# Fields are type-checked and converted to the declared types.
# For example, `mt.values` is converted to a Tensor.
print(mt.values)

O construtor aciona um erro de tipo `TypeError` se o valor de um campo não puder ser convertido no tipo declarado:

In [ ]:
try:
  MaskedTensor([1, 2, 3], None)
except TypeError as e:
  print(f"Got expected TypeError: {e}")

O valor padrão de um campo pode ser especificado definindo seu valor na classe:

In [ ]:
class Pencil(tf.experimental.ExtensionType):
  color: str = "black"
  has_erasor: bool = True
  length: tf.Tensor = 1.0

Pencil()

In [ ]:
Pencil(length=0.5, color="blue")

### Representação que pode ser exibida via print

`ExtensionType` adiciona um método de representação padrão que pode ser exibida via print (`__repr__`) que inclui o nome da classe e o valor de cada campo:


In [ ]:
print(MaskedTensor(values=[1, 2, 3], mask=[True, True, False]))

### Operadores de igualdade

`ExtensionType` adiciona os operadores de igualdade padrão (`__eq__` e `__ne__`), que consideram os dois valores iguais se eles tiverem o mesmo tipo e se todos os seus campos forem iguais. Os campos de tensor são considerados iguais se tiverem o mesmo formato e se todos os elementos forem iguais.

In [ ]:
a = MaskedTensor([1, 2], [True, False])
b = MaskedTensor([[3, 4], [5, 6]], [[False, True], [True, True]])
print(f"a == a: {a==a}")
print(f"a == b: {a==b}")
print(f"a == a.values: {a==a.values}")

**Observação:** se algum campo tiver um `Tensor`, então `__eq__` poderá retornar um `Tensor` booleano escalar (em vez de um valor booleano Python).

### Método de validação

`ExtensionType` adiciona um método `__validate__`, que pode ser sobrescrito para fazer verificações de validação dos campos. Ele é executado após a chamada ao construtor e depois que os campos tiverem o tipo verificado e tiverem sido convertidos nos tipos declarados para que possa pressupor que todos os campos estejam com os tipos declarados.

O exemplo abaixo atualiza `MaskedTensor` de forma a validar os `shape`s (formatos) e `dtype`s (tipos de dados) de seus campos.

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  """A tensor paired with a boolean mask, indicating which values are valid."""
  values: tf.Tensor
  mask: tf.Tensor
  def __validate__(self):
    self.values.shape.assert_is_compatible_with(self.mask.shape)
    assert self.mask.dtype.is_bool, 'mask.dtype must be bool'

In [ ]:
try:
  MaskedTensor([1, 2, 3], [0, 1, 0])  # Wrong `dtype` for mask.
except AssertionError as e:
  print(f"Got expected AssertionError: {e}")

In [ ]:
try:
  MaskedTensor([1, 2, 3], [True, False])  # shapes don't match.
except ValueError as e:
  print(f"Got expected ValueError: {e}")

### Imutabilidade imposta

`ExtensionType` sobrescreve os métodos `__setattr__` e `__delattr__` para evitar mudanças, garantido que os valores desse tipo de extensão sejam imutáveis.

In [ ]:
mt = MaskedTensor([1, 2, 3], [True, False, True])

In [ ]:
try:
  mt.mask = [True, True, True]
except AttributeError as e:
  print(f"Got expected AttributeError: {e}")

In [ ]:
try:
  mt.mask[0] = False
except TypeError as e:
  print(f"Got expected TypeError: {e}")

In [ ]:
try:
  del mt.mask
except AttributeError as e:
  print(f"Got expected AttributeError: {e}")

### TypeSpec aninhado

Cada classe `ExtensionType` tem uma classe `TypeSpec` correspondente, criada automaticamente e armazenada como `<extension_type_name>.Spec`.

Essa classe captura todas as informações de um valor, *exceto* os valores de tensores aninhados. Especificamente, o `TypeSpec` de um valor é criado substituindo-se qualquer Tensor, ExtensionType ou CompositeTensor aninhado por seu `TypeSpec`.


In [ ]:
class Player(tf.experimental.ExtensionType):
  name: tf.Tensor
  attributes: Mapping[str, tf.Tensor]

anne = Player("Anne", {"height": 8.3, "speed": 28.1})
anne_spec = tf.type_spec_from_value(anne)
print(anne_spec.name)  # Records `dtype` and `shape`, but not the string value.
print(anne_spec.attributes)  # Records keys and TensorSpecs for values.

Os valores de `TypeSpec` podem ser construídos explicitamente ou podem ser construídos a partir de um valor `ExtensionType` usando-se `tf.type_spec_from_value`:

In [ ]:
spec1 = Player.Spec(name=tf.TensorSpec([], tf.float32), attributes={})
spec2 = tf.type_spec_from_value(anne)

Os `TypeSpec`s são usados pelo TensorFlow para dividir valores em um **componente estático** e uma **componente dinâmico **:

- O **componente estático ** (fixado no momento da criação do grafo) é codificado com um `tf.TypeSpec`.
- O **componente dinâmico** (que pode variar a cada vez que o gráfico for criado) é codificado como uma lista de `tf.Tensor`s.

Por exemplo, `tf.function` faz o retracing de sua função encapsulada sempre que um argumento tiver um `TypeSpec` não visto anteriormente:

In [ ]:
@tf.function
def anonymize_player(player):
  print("<<TRACING>>")
  return Player("<anonymous>", player.attributes)

In [ ]:
# Function gets traced (first time the function has been called):
anonymize_player(Player("Anne", {"height": 8.3, "speed": 28.1}))

In [ ]:
# Function does NOT get traced (same TypeSpec: just tensor values changed)
anonymize_player(Player("Bart", {"height": 8.1, "speed": 25.3}))

In [ ]:
# Function gets traced (new TypeSpec: keys for attributes changed):
anonymize_player(Player("Chuck", {"height": 11.0, "jump": 5.3}))

Confira mais informações no [guia de tf.function](https://www.tensorflow.org/guide/function#rules_of_tracing).

## Personalização de `ExtensionType`s

Além de simplesmente declarar campos e seus tipos, os tipos de extensão podem:

- Sobrescrever a representação padrão que pode ser exibida via print (`__repr__`).
- Definir métodos.
- Definir `classmethod`s e `staticmethod`s.
- Definir propriedades.
- Sobrescrever o construtor padrão (`__init__`).
- Sobrescrever o operador de igualdade padrão (`__eq__`).
- Definir operadores (como `__add__` e `__lt__`).
- Declarar valores padrão para os campos.
- Definir subclasses.


### Como sobrescrever a representação padrão que pode ser exibida via print

É possível sobrescrever esse operador padrão de conversão de strings para os tipos de extensão. O exemplo abaixo atualiza a classe `MaskedTensor` de forma que ela gere uma representação de string mais fácil de ler quando os valores forem exibidos via print no modo adiantado (eager).

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  """A tensor paired with a boolean mask, indicating which values are valid."""
  values: tf.Tensor
  mask: tf.Tensor       # shape=values.shape; false for invalid values.

  def __repr__(self):
    return masked_tensor_str(self.values, self.mask)

def masked_tensor_str(values, mask):
  if isinstance(values, tf.Tensor):
    if hasattr(values, 'numpy') and hasattr(mask, 'numpy'):
      return f'<MaskedTensor {masked_tensor_str(values.numpy(), mask.numpy())}>'
    else:
      return f'MaskedTensor(values={values}, mask={mask})'
  if len(values.shape) == 1:
    items = [repr(v) if m else '_' for (v, m) in zip(values, mask)]
  else:
    items = [masked_tensor_str(v, m) for (v, m) in zip(values, mask)]
  return '[%s]' % ', '.join(items)

mt = MaskedTensor(values=[[1, 2, 3], [4, 5, 6]],
                  mask=[[True, True, False], [True, False, True]])
print(mt)

### Como definir métodos

Os tipos de extensão podem definir métodos como qualquer classe comum do Python. Por exemplo, o tipo `MaskedTensor` poderia definir um método `with_default` que retorne uma cópia de `self` com valores mascarados substituídos por um determinado valor `default`. Opcionalmente, é possível fazer uma anotação nos métodos com o decorador `@tf.function`.

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  values: tf.Tensor
  mask: tf.Tensor

  def with_default(self, default):
    return tf.where(self.mask, self.values, default)

MaskedTensor([1, 2, 3], [True, False, True]).with_default(0)

### Como definir `classmethod`s e `staticmethod`s

Os tipos de extensão podem definir métodos usando os decoradores `@classmethod` e `@staticmethod`. Por exemplo, o tipo `MaskedTensor` poderia definir um método de fábrica que mascare qualquer elemento com um determinado valor:

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  values: tf.Tensor
  mask: tf.Tensor

  def __repr__(self):
    return masked_tensor_str(self.values, self.mask)

  @staticmethod
  def from_tensor_and_value_to_mask(values, value_to_mask):
    return MaskedTensor(values, values != value_to_mask)

x = tf.constant([[1, 0, 2], [3, 0, 0]])
MaskedTensor.from_tensor_and_value_to_mask(x, 0)

### Como definir propriedades

Os tipos de extensão podem definir propriedades usando o decorador `@property` como qualquer classe comum do Python. Por exemplo, o tipo `MaskedTensor` poderia definir uma propriedade `dtype` que seja uma propriedade abreviada para o  `dtype` dos valores:

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  values: tf.Tensor
  mask: tf.Tensor

  @property
  def dtype(self):
    return self.values.dtype

MaskedTensor([1, 2, 3], [True, False, True]).dtype

### Como sobrescrever o construtor padrão

É possível sobrescrever o construtor padrão dos tipos de extensão. Os construtores personalizados precisam definir um valor para cada campo declarado e, após o construtor personalizado retornar, todos os campos terão seu tipo verificado, e os valores serão convertidos conforme descrito acima.

In [ ]:
class Toy(tf.experimental.ExtensionType):
  name: str
  price: tf.Tensor
  def __init__(self, name, price, discount=0):
    self.name = name
    self.price = price * (1 - discount)

print(Toy("ball", 5.0, discount=0.2))  # On sale -- 20% off!

Outra opção que você pode considerar é deixar o construtor padrão inalterado, mas adicionar um ou mais métodos de fábrica. Por exemplo:

In [ ]:
class Toy(tf.experimental.ExtensionType):
  name: str
  price: tf.Tensor

  @staticmethod
  def new_toy_with_discount(name, price, discount):
    return Toy(name, price * (1 - discount))

print(Toy.new_toy_with_discount("ball", 5.0, discount=0.2))

### Como sobrescrever o operador de igualdade padrão (`__eq__`)

É possível sobrescrever o operador padrão `__eq__` dos tipos de extensão. O exemplo abaixo atualiza `MaskedTensor` de modo a ignorar os elementos mascarados ao fazer a comparação de igualdade.

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  values: tf.Tensor
  mask: tf.Tensor

  def __repr__(self):
    return masked_tensor_str(self.values, self.mask)

  def __eq__(self, other):
    result = tf.math.equal(self.values, other.values)
    result = result | ~(self.mask & other.mask)
    return tf.reduce_all(result)

x = MaskedTensor([1, 2, 3, 4], [True, True, False, True])
y = MaskedTensor([5, 2, 0, 4], [False, True, False, True])
print(x == y)

**Observação:** geralmente, não é preciso sobrescrever `__ne__`, já que sua implementação padrão simplesmente chama `__eq__` e inverte o resultado.

### Como usar referências posteriores

Se o tipo de um campo ainda não tiver sido definido, você pode usar uma string contendo o nome do tipo. No exemplo abaixo, a string `"Node"` (nó) é usada para fazer uma anotação no campo `children` (filhos), pois o tipo  `Node` ainda não foi totalmente definido.


In [ ]:
class Node(tf.experimental.ExtensionType):
  value: tf.Tensor
  children: Tuple["Node", ...] = ()

Node(3, [Node(5), Node(2)])

### Como definir subclasses

É possível criar subclasses dos tipos de extensão usando a sintaxe padrão do Python. As subclasses de tipos de extensão podem adicionar novos campos, métodos e propriedades; podem também sobrescrever o construtor, a representação que pode ser exibida via print e o operador de qualidade. O exemplo abaixo define uma classe `TensorGraph` básica que usa três campos `Tensor` para codificar um conjunto de bordas entre nós. Em seguida, define uma subclasse que adiciona um campo `Tensor` para registrar um "valor de característica" para cada nó. A subclasse também define um método para propagar os valores de características para as bordas.

In [ ]:
class TensorGraph(tf.experimental.ExtensionType):
  num_nodes: tf.Tensor
  edge_src: tf.Tensor   # edge_src[e] = index of src node for edge e.
  edge_dst: tf.Tensor   # edge_dst[e] = index of dst node for edge e.

class TensorGraphWithNodeFeature(TensorGraph):
  node_features: tf.Tensor  # node_features[n] = feature value for node n.

  def propagate_features(self, weight=1.0) -> 'TensorGraphWithNodeFeature':
    updates = tf.gather(self.node_features, self.edge_src) * weight
    new_node_features = tf.tensor_scatter_nd_add(
        self.node_features, tf.expand_dims(self.edge_dst, 1), updates)
    return TensorGraphWithNodeFeature(
        self.num_nodes, self.edge_src, self.edge_dst, new_node_features)

g = TensorGraphWithNodeFeature(  # Edges: 0->1, 4->3, 2->2, 2->1
    num_nodes=5, edge_src=[0, 4, 2, 2], edge_dst=[1, 3, 2, 1],
    node_features=[10.0, 0.0, 2.0, 5.0, -1.0, 0.0])

print("Original features:", g.node_features)
print("After propagating:", g.propagate_features().node_features)

### Como definir campos privados

Os campos de um tipo de extensão podem ser marcados como privados adicionando um sublinhado como prefixo (seguindo as convenções padrão do Python). Isso não impacta a forma como o TensorFlow trata os campos, apenas serve como um sinal para qualquer usuário do tipo da extensão de que esses campos são privados.


### Personalização do `TypeSpec` de `ExtensionType`

Cada classe `ExtensionType` tem uma classe `TypeSpec` correspondente, criada automaticamente e armazenada como `<extension_type_name>.Spec`. Confira mais informações na seção "TypeSpec aninhado" acima.

Para personalizar o `TypeSpec`, basta definir sua própria classe aninhada chamada `Spec`, e `ExtensionType` vai usá-la como base para o `TypeSpec` construído automaticamente. Você pode personalizar a classe `Spec` ao:

- Sobrescrever a representação padrão que pode ser exibida via print.
- Sobrescrever o construtor padrão.
- Definir métodos, `classmethod`s, `staticmethod`s e propriedades.

O exemplo abaixo personaliza a classe `MaskedTensor.Spec` para facilitar o uso:

In [ ]:
class MaskedTensor(tf.experimental.ExtensionType):
  values: tf.Tensor
  mask: tf.Tensor

  shape = property(lambda self: self.values.shape)
  dtype = property(lambda self: self.values.dtype)

  def __repr__(self):
    return masked_tensor_str(self.values, self.mask)

  def with_values(self, new_values):
    return MaskedTensor(new_values, self.mask)

  class Spec:
    def __init__(self, shape, dtype=tf.float32):
      self.values = tf.TensorSpec(shape, dtype)
      self.mask = tf.TensorSpec(shape, tf.bool)

    def __repr__(self):
      return f"MaskedTensor.Spec(shape={self.shape}, dtype={self.dtype})"

    shape = property(lambda self: self.values.shape)
    dtype = property(lambda self: self.values.dtype)

**Observação**: a classe `Spec` personalizada não pode usar variáveis de instância que não tenham sido declaradas no `ExtensionType` original.

## Dispatch de API de tensor

Os tipos de extensão podem ser do "tipo tensor", o que significa que podem especializar ou estender a interface definida pelo tipo `tf.Tensor`. Entre os exemplos desses tipos de extensão "tipo tensor" estão `RaggedTensor`, `SparseTensor` e `MaskedTensor`. ***Decoradores de dispatch*** podem ser usados para sobrescrever o comportamento padrão das operações do TensorFlow quando aplicados aos tipos de extensão "tipo tensor". Atualmente, o TensorFlow define três decoradores de dispatch:

- `@tf.experimental.dispatch_for_api(tf_api)`
- `@tf.experimental.dispatch_for_unary_elementwise_apis(x_type)`
- `@tf.experimental.dispatch_for_binary_elementwise_apis(x_type, y_type)`

### Dispatch para uma única API

O decorador `tf.experimental.dispatch_for_api` sobrescreve o comportamento padrão de uma operação do TensorFlow especificada quando é chamado com a assinatura especificada. Por exemplo, é possível usar esse decorador para especificar como `tf.stack` deve processar os valores de `MaskedTensor`:

In [ ]:
@tf.experimental.dispatch_for_api(tf.stack)
def masked_stack(values: List[MaskedTensor], axis = 0):
  return MaskedTensor(tf.stack([v.values for v in values], axis),
                      tf.stack([v.mask for v in values], axis))

Isso sobrescreve a implementação padrão de `tf.stack` sempre que é chamado com uma lista de valores de `MaskedTensor` (já que o argumento `values` tem a anotação `typing.List[MaskedTensor]`):

In [ ]:
x = MaskedTensor([1, 2, 3], [True, True, False])
y = MaskedTensor([4, 5, 6], [False, True, True])
tf.stack([x, y])

Para permitir que `tf.stack` lide com listas de valores misturados de `MaskedTensor` e `Tensor`, você pode refinar a anotação de tipo do parâmetro `values` e atualizar o corpo da função:

In [ ]:
tf.experimental.unregister_dispatch_for(masked_stack)

def convert_to_masked_tensor(x):
  if isinstance(x, MaskedTensor):
    return x
  else:
    return MaskedTensor(x, tf.ones_like(x, tf.bool))

@tf.experimental.dispatch_for_api(tf.stack)
def masked_stack_v2(values: List[Union[MaskedTensor, tf.Tensor]], axis = 0):
  values = [convert_to_masked_tensor(v) for v in values]
  return MaskedTensor(tf.stack([v.values for v in values], axis),
                      tf.stack([v.mask for v in values], axis))
x = MaskedTensor([1, 2, 3], [True, True, False])
y = tf.constant([4, 5, 6])
tf.stack([x, y, x])

Confira uma lista de APIs que podem ser sobrescritas na documentação da API de `tf.experimental.dispatch_for_api`.

### Dispatch para todas as APIs elemento a elemento unárias

O decorador `tf.experimental.dispatch_for_unary_elementwise_apis` sobrescreve o comportamento padrão de ***todas*** as operações elemento a elemento unárias (como `tf.math.cos`) sempre que o valor do primeiro argumento (geralmente chamado de `x`) corresponde à anotação de tipo `x_type`. A função decorada deve receber dois argumentos:

- `api_func`: função que recebe um único parâmetro e faz a operação elemento a elemento (por exemplo, `tf.abs`).
- `x`: primeiro argumento da operação elemento a elemento.

O exemplo abaixo atualiza todas as operações elemento a elemento unárias de modo a lidarem com o tipo `MaskedTensor`:

In [ ]:
 @tf.experimental.dispatch_for_unary_elementwise_apis(MaskedTensor)
 def masked_tensor_unary_elementwise_api_handler(api_func, x):
   return MaskedTensor(api_func(x.values), x.mask)

Agora, essa função será usada sempre que uma operação elemento a elemento unária seja chamada em um `MaskedTensor`.

In [ ]:
 x = MaskedTensor([1, -2, -3], [True, False, True])
 print(tf.abs(x))

In [ ]:
print(tf.ones_like(x, dtype=tf.float32))

### Dispatch para todas as APIs elemento a elemento binárias

De maneira similar, `tf.experimental.dispatch_for_binary_elementwise_apis` pode ser usado para atualizar todas as operações elemento a elemento binárias de modo a lidarem com o tipo `MaskedTensor`:


In [ ]:
@tf.experimental.dispatch_for_binary_elementwise_apis(MaskedTensor, MaskedTensor)
def masked_tensor_binary_elementwise_api_handler(api_func, x, y):
  return MaskedTensor(api_func(x.values, y.values), x.mask & y.mask)

In [ ]:
x = MaskedTensor([1, -2, -3], [True, False, True])
y = MaskedTensor([[4], [5]], [[True], [False]])
tf.math.add(x, y)

Confira uma lista das APIs elemento a elemento que podem ser sobrescritas na documentação da API de `tf.experimental.dispatch_for_unary_elementwise_apis` e de `tf.experimental.dispatch_for_binary_elementwise_apis`.

## `ExtensionType`s que podem ser divididos em lotes

Um `ExtensionType` *pode ser dividido em lotes* se uma única instância puder ser usada para representar um lote de valores. Geralmente, é possível fazer isso adicionando dimensões de lote a todos os `Tensor`s aninhados. As APIs do TensorFlow abaixo exigem que qualquer entrada de tipo de extensão possa ser dividida em lotes:

- `tf.data.Dataset` (`batch`, `unbatch`, `from_tensor_slices`)
- `tf.keras` (`fit`, `evaluate`, `predict`)
- `tf.map_fn`

Por padrão, `BatchableExtensionType` cria valores em lotes fazendo a divisão em lotes de qualquer `Tensor`, `CompositeTensor` e `ExtensionType` aninhado. Se isso não for adequado para sua classe, você precisará usar `tf.experimental.ExtensionTypeBatchEncoder` para sobrescrever esse comportamento padrão. Por exemplo, não seria apropriado criar um lote de valores de `tf.SparseTensor` simplesmente empilhando os campos `values`, `indices` e `dense_shape` de tensores esparsos individuais. Na maioria dos casos, você não pode empilhar esses tensores, pois eles têm formatos incompatíveis e, mesmo se você pudesse, o resultado não seria um `SparseTensor` válido.

**Observação**: os `BatchableExtensionType`s *não* definem automaticamente dispatchers para `tf.stack`, `tf.concat`, `tf.slice`, etc. Se a sua classe precisar ter suporte a essas APIs, use os decoradores de dispatch descritos acima.

### Exemplo de `BatchableExtensionType`: `Network`

Como exemplo, considere uma classe `Network` usada para balanceamento de carga que monitore quanto trabalho falta fazer em cada nó e quanta largura de banda está disponível para movimentar entre os nós:

In [ ]:
class Network(tf.experimental.ExtensionType):  # This version is not batchable.
  work: tf.Tensor       # work[n] = work left to do at node n
  bandwidth: tf.Tensor  # bandwidth[n1, n2] = bandwidth from n1->n2

net1 = Network([5., 3, 8], [[0., 2, 0], [2, 0, 3], [0, 3, 0]])
net2 = Network([3., 4, 2], [[0., 2, 2], [2, 0, 2], [2, 2, 0]])

Para que esse tipo possa ser dividido em lotes, altere o tipo base para `BatchableExtensionType` e ajuste o formato de cada campo de forma a incluir dimensões de lote opcionais. O exemplo abaixo também adiciona um campo `shape` para controlar o formato do lote. Esse campo `shape` não é exigido por `tf.data.Dataset` ou por `tf.map_fn`, mas *é* exigido por `tf.keras`.

In [ ]:
class Network(tf.experimental.BatchableExtensionType):
  shape: tf.TensorShape  # batch shape. A single network has shape=[].
  work: tf.Tensor        # work[*shape, n] = work left to do at node n
  bandwidth: tf.Tensor   # bandwidth[*shape, n1, n2] = bandwidth from n1->n2

  def __init__(self, work, bandwidth):
    self.work = tf.convert_to_tensor(work)
    self.bandwidth = tf.convert_to_tensor(bandwidth)
    work_batch_shape = self.work.shape[:-1]
    bandwidth_batch_shape = self.bandwidth.shape[:-2]
    self.shape = work_batch_shape.merge_with(bandwidth_batch_shape)

  def __repr__(self):
    return network_repr(self)

def network_repr(network):
  work = network.work
  bandwidth = network.bandwidth
  if hasattr(work, 'numpy'):
    work = ' '.join(str(work.numpy()).split())
  if hasattr(bandwidth, 'numpy'):
    bandwidth = ' '.join(str(bandwidth.numpy()).split())
  return (f"<Network shape={network.shape} work={work} bandwidth={bandwidth}>")

In [ ]:
net1 = Network([5., 3, 8], [[0., 2, 0], [2, 0, 3], [0, 3, 0]])
net2 = Network([3., 4, 2], [[0., 2, 2], [2, 0, 2], [2, 2, 0]])
batch_of_networks = Network(
    work=tf.stack([net1.work, net2.work]),
    bandwidth=tf.stack([net1.bandwidth, net2.bandwidth]))
print(f"net1={net1}")
print(f"net2={net2}")
print(f"batch={batch_of_networks}")

Em seguida, você pode usar `tf.data.Dataset` para fazer a iteração de um lote de redes:

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(batch_of_networks)
for i, network in enumerate(dataset):
  print(f"Batch element {i}: {network}")

E também pode usar `map_fn` para aplicar uma função a cada elemento dos lotes:

In [ ]:
def balance_work_greedy(network):
  delta = (tf.expand_dims(network.work, -1) - tf.expand_dims(network.work, -2))
  delta /= 4
  delta = tf.maximum(tf.minimum(delta, network.bandwidth), -network.bandwidth)
  new_work = network.work + tf.reduce_sum(delta, -1)
  return Network(new_work, network.bandwidth)

tf.map_fn(balance_work_greedy, batch_of_networks)

## APIs do TensorFlow com suporte aos `ExtensionType`s

### @tf.function

[`tf.function`](https://www.tensorflow.org/guide/function) é um decorador que pré-computa os grafos do TensorFlow para as funções em Python, o que pode melhorar significativamente o desempenho do seu código do TensorFlow. Os valores de tipo de extensão podem ser usados de maneira transparente com funções `@tf.function` decoradas.

In [ ]:
class Pastry(tf.experimental.ExtensionType):
  sweetness: tf.Tensor  # 2d embedding that encodes sweetness
  chewiness: tf.Tensor  # 2d embedding that encodes chewiness

@tf.function
def combine_pastry_features(x: Pastry):
  return (x.sweetness + x.chewiness) / 2

cookie = Pastry(sweetness=[1.2, 0.4], chewiness=[0.8, 0.2])
combine_pastry_features(cookie)

Se você quiser especificar `input_signature` explicitamente para `tf.function`, pode fazer isso usando o `TypeSpec` do tipo de extensão.

In [ ]:
pastry_spec = Pastry.Spec(tf.TensorSpec([2]), tf.TensorSpec(2))

@tf.function(input_signature=[pastry_spec])
def increase_sweetness(x: Pastry, delta=1.0):
  return Pastry(x.sweetness + delta, x.chewiness)

increase_sweetness(cookie)

#### Funções concretas

As funções concretas encapsulam os grafos traçados individuais que são criados por `tf.function`. Os tipos de extensão podem ser usados de maneira transparente com funções concretas.


In [ ]:
cf = combine_pastry_features.get_concrete_function(pastry_spec)
cf(cookie)

### Operações de fluxo de controle

As operações de fluxo de controle do TensorFlow têm suporte aos tipos de extensão:

- `tf.cond`
- `tf.case`
- `tf.while_loop`
- `tf.identity`


In [ ]:
# Example: using tf.cond to select between two MaskedTensors. Note that the
# two MaskedTensors don't need to have the same shape.
a = MaskedTensor([1., 2, 3], [True, False, True])
b = MaskedTensor([22., 33, 108, 55], [True, True, True, False])
condition = tf.constant(True)
print(tf.cond(condition, lambda: a, lambda: b))

In [ ]:
# Example: using tf.while_loop with MaskedTensor.
cond = lambda i, _: i < 10
def body(i, mt):
  return i + 1, mt.with_values(mt.values + 3 / 7)
print(tf.while_loop(cond, body, [0, b])[1])

### Fluxo de controle do Autograph

As declarações de fluxo de controle em `tf.function` também têm suporte aos tipos de extensão (usando o Autograph). No exemplo abaixo, a declaração `if` e as declarações `for` são convertidas automaticamente em operações `tf.cond` e `tf.while_loop`, que têm suporte aos tipos de extensão.

In [ ]:
@tf.function
def fn(x, b):
  if b:
    x = MaskedTensor(x, tf.less(x, 0))
  else:
    x = MaskedTensor(x, tf.greater(x, 0))
  for i in tf.range(5 if b else 7):
    x = x.with_values(x.values + 1 / 2)
  return x

print(fn(tf.constant([1., -2, 3]), tf.constant(True)))
print(fn(tf.constant([1., -2, 3]), tf.constant(False)))

### Keras

[tf.keras](https://www.tensorflow.org/guide/keras) é a API de alto nível do TensorFlow para criar e treinar modelos de aprendizado profundo. Os tipos de extensão podem ser passados como entradas para um modelo do Keras, passados entre camadas do Keras e retornados por modelos do Keras. Atualmente, o Keras tem dois requisitos para os tipos de extensão:

- Eles precisam poder ser transformados em lotes (confira a seção "`ExtensionType`s que podem ser divididos em lote" acima).
- Eles precisam ter um campo ou propriedade chamado `shape`. Pressupõe-se que `shape[0]` seja a dimensão de lote.

As duas subseções abaixo fornecem exemplos que mostram como os tipos de extensão podem ser usados com o Keras.


#### Exemplo do Keras: `Network`

No primeiro exemplo, considere a classe `Network` (Rede) definida na seção "`ExtensionType`s que podem ser divididos em lotes" acima, que pode ser usada para balanceamento de carga de trabalho entre nós. Repetimos a definição da classe aqui:

In [ ]:
class Network(tf.experimental.BatchableExtensionType):
  shape: tf.TensorShape  # batch shape. A single network has shape=[].
  work: tf.Tensor        # work[*shape, n] = work left to do at node n
  bandwidth: tf.Tensor   # bandwidth[*shape, n1, n2] = bandwidth from n1->n2

  def __init__(self, work, bandwidth):
    self.work = tf.convert_to_tensor(work)
    self.bandwidth = tf.convert_to_tensor(bandwidth)
    work_batch_shape = self.work.shape[:-1]
    bandwidth_batch_shape = self.bandwidth.shape[:-2]
    self.shape = work_batch_shape.merge_with(bandwidth_batch_shape)

  def __repr__(self):
    return network_repr(self)

In [ ]:
single_network = Network(  # A single network with 4 nodes.
    work=[8.0, 5, 12, 2],
    bandwidth=[[0.0, 1, 2, 2], [1, 0, 0, 2], [2, 0, 0, 1], [2, 2, 1, 0]])

batch_of_networks = Network(  # Batch of 2 networks, each w/ 2 nodes.
    work=[[8.0, 5], [3, 2]],
    bandwidth=[[[0.0, 1], [1, 0]], [[0, 2], [2, 0]]])

Você pode definir uma nova camada do Keras que processe `Network`s.

In [ ]:
class BalanceNetworkLayer(tf.keras.layers.Layer):
  """Layer that balances work between nodes in a network.

  Shifts work from more busy nodes to less busy nodes, constrained by bandwidth.
  """
  def call(self, inputs):
    # This function is defined above in the "Batchable `ExtensionType`s" section.
    return balance_work_greedy(inputs)

Em seguida, você pode usar essas camadas para criar um modelo simples. Para alimentar um `ExtensionType` em um modelo, você pode usar uma camada `tf.keras.layer.Input` com `type_spec` definido como o `TypeSpec` do tipo de extensão. Se o modelo do Keras for usado para processar lotes, então `type_spec` precisa incluir a dimensão de lote.

In [ ]:
input_spec = Network.Spec(shape=None,
                          work=tf.TensorSpec(None, tf.float32),
                          bandwidth=tf.TensorSpec(None, tf.float32))
model = tf.keras.Sequential([
    tf.keras.layers.Input(type_spec=input_spec),
    BalanceNetworkLayer(),
    ])

Por fim, você pode aplicar o modelo a uma única rede e a um lote de redes.

In [ ]:
model(single_network)

In [ ]:
model(batch_of_networks)

#### Exemplo do Keras: MaskedTensor

Neste exemplo, `MaskedTensor` é estendido para ter suporte ao `Keras`. `shape` é definido como uma propriedade calculada a partir do campo `values`. O Keras exige que você adicione essa propriedade tanto ao tipo de extensão quanto ao seu `TypeSpec`. `MaskedTensor` também define uma variável `__name__`, que será exigida para a serialização do `SavedModel` (abaixo).

In [ ]:
class MaskedTensor(tf.experimental.BatchableExtensionType):
  # __name__ is required for serialization in SavedModel; see below for details.
  __name__ = 'extension_type_colab.MaskedTensor'

  values: tf.Tensor
  mask: tf.Tensor

  shape = property(lambda self: self.values.shape)
  dtype = property(lambda self: self.values.dtype)

  def with_default(self, default):
    return tf.where(self.mask, self.values, default)

  def __repr__(self):
    return masked_tensor_str(self.values, self.mask)

  class Spec:
    def __init__(self, shape, dtype=tf.float32):
      self.values = tf.TensorSpec(shape, dtype)
      self.mask = tf.TensorSpec(shape, tf.bool)

    shape = property(lambda self: self.values.shape)
    dtype = property(lambda self: self.values.dtype)

    def with_shape(self):
      return MaskedTensor.Spec(tf.TensorSpec(shape, self.values.dtype),
                               tf.TensorSpec(shape, self.mask.dtype))

Em seguida, os decoradores de dispatch são usados para sobrescrever o comportamento padrão de diversas APIs do TensorFlow. Como essas APIs são usadas por camadas padrão do Keras (como a camada `Dense`), sobrescrevê-la permitirá que usemos essas camadas com `MaskedTensor`. Para a finalidade deste exemplo, o `matmul` para os tensores mascarados é definido de forma a tratar os valores mascarados como zero (isto é, eles não são incluídos no produto).

In [ ]:
@tf.experimental.dispatch_for_unary_elementwise_apis(MaskedTensor)
def unary_elementwise_op_handler(op, x):
 return MaskedTensor(op(x.values), x.mask)

@tf.experimental.dispatch_for_binary_elementwise_apis(
    Union[MaskedTensor, tf.Tensor],
    Union[MaskedTensor, tf.Tensor])
def binary_elementwise_op_handler(op, x, y):
  x = convert_to_masked_tensor(x)
  y = convert_to_masked_tensor(y)
  return MaskedTensor(op(x.values, y.values), x.mask & y.mask)

@tf.experimental.dispatch_for_api(tf.matmul)
def masked_matmul(a: MaskedTensor, b,
                  transpose_a=False, transpose_b=False,
                  adjoint_a=False, adjoint_b=False,
                  a_is_sparse=False, b_is_sparse=False,
                  output_type=None):
  if isinstance(a, MaskedTensor):
    a = a.with_default(0)
  if isinstance(b, MaskedTensor):
    b = b.with_default(0)
  return tf.matmul(a, b, transpose_a, transpose_b, adjoint_a,
                   adjoint_b, a_is_sparse, b_is_sparse, output_type)

Em seguida, você pode criar um modelo do Keras que aceite `MaskedTensor` como entrada usando as camadas padrão do Keras:

In [ ]:
input_spec = MaskedTensor.Spec([None, 2], tf.float32)

masked_tensor_model = tf.keras.Sequential([
    tf.keras.layers.Input(type_spec=input_spec),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1)])
masked_tensor_model.compile(loss='binary_crossentropy', optimizer='rmsprop')

In [ ]:
a = MaskedTensor([[1., 2], [3, 4], [5, 6]],
                  [[True, False], [False, True], [True, True]])
masked_tensor_model.fit(a, tf.constant([[1], [0], [1]]), epochs=3)
print(masked_tensor_model(a))

### SavedModel

Um [SavedModel](https://www.tensorflow.org/guide/saved_model) é um programa do TensorFlow serializado, incluindo tanto os pesos quanto a computação. Ele pode ser criado a partir de um modelo do Keras ou de um modelo personalizado. Nos dois casos, os tipos de extensão podem ser usados de maneira transparente com as funções e os métodos definidos por um SavedModel.

O SavedModel pode salvar modelos, camadas e funções que processem tipos de extensão, desde que os tipos de extensão tenham um `__name__`. Esse nome é usado para registrar o tipo de extensão para que ele possa ser localizado quando o modelo for carregado.

#### Exemplo: salvando um modelo do Keras

Os modelos do Keras que usam tipos de extensão podem ser salvos utilizando-se `SavedModel`.

In [ ]:
masked_tensor_model_path = tempfile.mkdtemp()
tf.saved_model.save(masked_tensor_model, masked_tensor_model_path)
imported_model = tf.saved_model.load(masked_tensor_model_path)
imported_model(a)

#### Exemplo: salvando um modelo personalizado

O SavedModel também pode ser usado para salvar subclasses `tf.Module` personalizadas com funções que processem tipos de extensão.

In [ ]:
class CustomModule(tf.Module):
  def __init__(self, variable_value):
    super().__init__()
    self.v = tf.Variable(variable_value)

  @tf.function
  def grow(self, x: MaskedTensor):
    """Increase values in `x` by multiplying them by `self.v`."""
    return MaskedTensor(x.values * self.v, x.mask)

module = CustomModule(100.0)

module.grow.get_concrete_function(MaskedTensor.Spec(shape=None,
                                                    dtype=tf.float32))
custom_module_path = tempfile.mkdtemp()
tf.saved_model.save(module, custom_module_path)
imported_model = tf.saved_model.load(custom_module_path)
imported_model.grow(MaskedTensor([1., 2, 3], [False, True, False]))

#### Como carregar um SavedModel quando o `ExtensionType` estiver indisponível

Se você carregar um `SavedModel` que use um `ExtensionType`, mas esse `ExtensionType` não estiver disponível (ou seja, não foi importado), será exibido um aviso, e o TensorFlow usará um objeto de "tipo de extensão anônimo". Esse objeto terá os mesmos campos que o tipo original, mas não terá qualquer outra personalização que você tenha adicionado ao tipo, como métodos ou propriedades personalizados.

#### Como usar `ExtensionType`s com o TensorFlow Serving

No momento, o [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving) (e todos os outros consumidores do dicionário de "assinaturas" do SavedModel) exigem que todas as entradas e saídas sejam tensores brutos. Se você quiser usar o TensorFlow Serving com um modelo que tenha tipos de extensão, pode adicionar métodos encapsuladores que façam a composição ou decomposição dos valores de tipo de extensão dos tensores. Por exemplo:

In [ ]:
class CustomModuleWrapper(tf.Module):
  def __init__(self, variable_value):
    super().__init__()
    self.v = tf.Variable(variable_value)

  @tf.function
  def var_weighted_mean(self, x: MaskedTensor):
    """Mean value of unmasked values in x, weighted by self.v."""
    x = MaskedTensor(x.values * self.v, x.mask)
    return (tf.reduce_sum(x.with_default(0)) /
            tf.reduce_sum(tf.cast(x.mask, x.dtype)))

  @tf.function()
  def var_weighted_mean_wrapper(self, x_values, x_mask):
    """Raw tensor wrapper for var_weighted_mean."""
    return self.var_weighted_mean(MaskedTensor(x_values, x_mask))

module = CustomModuleWrapper([3., 2., 8., 5.])

module.var_weighted_mean_wrapper.get_concrete_function(
    tf.TensorSpec(None, tf.float32), tf.TensorSpec(None, tf.bool))
custom_module_path = tempfile.mkdtemp()
tf.saved_model.save(module, custom_module_path)
imported_model = tf.saved_model.load(custom_module_path)
x = MaskedTensor([1., 2., 3., 4.], [False, True, False, True])
imported_model.var_weighted_mean_wrapper(x.values, x.mask)

### `Dataset`s

[`tf.data`](https://www.tensorflow.org/guide/data) é uma API que permite criar pipelines de entrada complexos a partir de partes simples e reutilizáveis. A estrutura de dados principal é `tf.data.Dataset`, que representa uma sequência de elementos, onde cada um consiste em um ou mais componentes.

#### Como criar `Dataset`s com tipos de extensão

É possível criar datasets usando valores de tipo de extensão usando `Dataset.from_tensors`, `Dataset.from_tensor_slices` ou `Dataset.from_generator`:

In [ ]:
ds = tf.data.Dataset.from_tensors(Pastry(5, 5))
iter(ds).next()

In [ ]:
mt = MaskedTensor(tf.reshape(range(20), [5, 4]), tf.ones([5, 4]))
ds = tf.data.Dataset.from_tensor_slices(mt)
for value in ds:
  print(value)

In [ ]:
def value_gen():
  for i in range(2, 7):
    yield MaskedTensor(range(10), [j%i != 0 for j in range(10)])

ds = tf.data.Dataset.from_generator(
    value_gen, output_signature=MaskedTensor.Spec(shape=[10], dtype=tf.int32))
for value in ds:
  print(value)

#### Como criar e separar lotes de `Dataset`s com tipos de extensão

É possível criar e separar lotes de datasets com tipos de extensão usando `Dataset.batch` e `Dataset.unbatch`.

In [ ]:
batched_ds = ds.batch(2)
for value in batched_ds:
  print(value)

In [ ]:
unbatched_ds = batched_ds.unbatch()
for value in unbatched_ds:
  print(value)